Notebook08 Stacking Kernel structure 

Timeline: 2017/11/8

Goal: Look at Kernels online and develop a structure for stacking a large number of kernels

I. Import Packages, define functions and import files

In [1]:
# import packages
# Starting time: 12:07

# Data Manipulation
import pandas as pd
import numpy as np

# Training packages
import xgboost as xgb
import lightgbm as lgb
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

# Ensemble
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier

# Cross Validation
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import gc

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:261: DeprecationWarning: invalid escape sequence \s
  """
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:379: DeprecationWarning: invalid escape sequence \s
  """
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1660: DeprecationWarning: invalid escape sequence \s
  """
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1844: DeprecationWarning: invalid escape sequence \s
  """
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:2017: DeprecationWarning: invalid escape sequence \s
  """
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:2161: DeprecationWarning: invalid escape sequence \s
  """
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\omp.py:349: DeprecationWarning: invalid escape sequence \g
  """


In [2]:
# Functions that might be useful

# 01 Gini coefficient calculations
# https://www.kaggle.com/c/ClaimPredictionChallenge/discussion/703
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
 
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

# 02 Gini coefficient for xgb and lgb
# https://www.kaggle.com/rshally/porto-xgb-lgb-kfold-lb-0-282  (Version 1)
def gini_xgb(pred, y):
    y = y.get_label()
    return 'gini', gini(y, pred) / gini(y, y)

def gini_lgb(preds, dtrain):
    y = list(dtrain.get_label())
    score = gini(y, preds) / gini(y, y)
    return 'gini', score, True

# 03 The Ensemble function
# https://www.kaggle.com/yekenot/simple-stacker-lb-0-284/code (Version 8)
class Ensemble(object):
    def __init__(self, n_splits, stacker, base_models):
        self.n_splits = n_splits
        self.stacker = stacker
        self.base_models = base_models

    def fit_predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)

        folds = list(StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=2016).split(X, y))

        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))
        for i, clf in enumerate(self.base_models):

            S_test_i = np.zeros((T.shape[0], self.n_splits))

            for j, (train_idx, test_idx) in enumerate(folds):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
#                y_holdout = y[test_idx]

                print ("Fit %s fold %d" % (str(clf).split('(')[0], j+1))
                clf.fit(X_train, y_train)
#                cross_score = cross_val_score(clf, X_train, y_train, cv=3, scoring='roc_auc')
#                print("    cross_score: %.5f" % (cross_score.mean()))
                y_pred = clf.predict_proba(X_holdout)[:,1]                

                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = clf.predict_proba(T)[:,1]
            S_test[:, i] = S_test_i.mean(axis=1)

        results = cross_val_score(self.stacker, S_train, y, cv=3, scoring='roc_auc')
        print("Stacker score: %.5f" % (results.mean()))

        self.stacker.fit(S_train, y)
        res = self.stacker.predict_proba(S_test)[:,1]
        return res

In [3]:
# Loading Files and Picking out NA values
# It seems that if we don't pick out NA values, there will be one missing value in the id column
print('loading files...')
train = pd.read_csv('/Users/maxji/Desktop/Kaggle/0SafeDriver/data/train.csv', na_values=-1)
test = pd.read_csv('/Users/maxji/Desktop/Kaggle/0SafeDriver/data/test.csv', na_values=-1)

# Change format to reduce memory usage
for c in train.select_dtypes(include=['float64']).columns:
    train[c]=train[c].astype(np.float32)
    test[c]=test[c].astype(np.float32)
for c in train.select_dtypes(include=['int64']).columns[2:]:
    train[c]=train[c].astype(np.int8)
    test[c]=test[c].astype(np.int8)  

# Print out the shape of train and test
print(train.shape,test.shape)

loading files...
(595212, 59) (892816, 58)


II. Data Manipulation

In [4]:
# 01 Dropping Columns starting with Calc
# Note: This is used by almost all kernels online. 
# Justification: https://www.kaggle.com/arthurtok/interactive-porto-insights-a-plot-ly-tutorial
# At least in Gradient Boosting, the calc variables all show really low correlation with target.

col_to_drop = train.columns[train.columns.str.startswith('ps_calc_')]
train = train.drop(col_to_drop, axis=1)  
test = test.drop(col_to_drop, axis=1) 

# 02 Treating missing values:
# Again, different ways are employed by different Kernels.
# a. Kernels that doesn't treat values (Keep NA in the data)
# https://www.kaggle.com/rshally/porto-xgb-lgb-kfold-lb-0-282  (Version 1)
# b. Kernels that keep NA values as -1
# c. Kernels that change NA values to 999/-999

# 03 Dealing with categorical variables
# a. Make up dummy variables for each of them
# Note: This will increase the number of variables, so need more parameter tuning
"""cat_features = [a for a in train.columns if a.endswith('cat')]
for column in cat_features:
    temp = pd.get_dummies(pd.Series(train[column]))
    train = pd.concat([train,temp],axis=1)
    train = train.drop([column],axis=1)
    
for column in cat_features:
    temp = pd.get_dummies(pd.Series(test[column]))
    test = pd.concat([test,temp],axis=1)
    test = test.drop([column],axis=1)

print(train.shape,test.shape)"""

(595212, 200) (892816, 199)


In [5]:
train.describe()

,id,target,ps_ind_01,ps_ind_03,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,...,95,96,97,98,99,100,101,102,103,104
count,5.952120e+05,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,...,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000
mean,7.438036e+05,0.036448,1.900378,4.423318,0.393742,0.257033,0.163921,0.185304,0.000373,0.001692,...,0.005978,0.003483,0.002493,0.004788,0.020231,0.007468,0.012330,0.003533,0.040762,0.142946
std,4.293678e+05,0.187401,1.983789,2.699902,0.488579,0.436998,0.370205,0.388544,0.019309,0.041097,...,0.077084,0.058912,0.049870,0.069031,0.140791,0.086094,0.110354,0.059336,0.197738,0.350018
min,7.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.719915e+05,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.435475e+05,0.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.115549e+06,0.000000,3.000000,6.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.488027e+06,1.000000,7.000000,11.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


III. Training

In [6]:
# Data Preparation for Training
# 01 Dropping columns, separating target function (y) and features (X)
#  X: The values of feature dataframe
#  y: target dataframe
#  features: The columns of feature dataframe
X = train.drop(['id', 'target'], axis=1)
features = X.columns
X = X.values
y = train['target'].values

# 02 Create and prepare the submission dataset
#  sub: The submission dataframe 
sub=test['id'].to_frame()
sub['target']=0

In [7]:
# Training parameters 
# 01 xgboost
# https://www.kaggle.com/rshally/porto-xgb-lgb-kfold-lb-0-282   (Version 1)
"""params = {'eta': 0.02, 'max_depth': 4, 'subsample': 0.9, 'colsample_bytree': 0.9, 
          'objective': 'binary:logistic', 'eval_metric': 'auc', 'silent': True}"""
# local cv: 0.2848016, lb: 0.281


# 02 lightgbm
# https://www.kaggle.com/rshally/porto-xgb-lgb-kfold-lb-0-282   (Version 1)

# @鲲(China) lgbm is very sensitive with hyper parameters, my lgbm give me 0.281. Here's my suggestion, 
# use a small max_depth and a num_of_leaves smaller than 2**max_depth, also try bagging with a small bagging frequency
params = {'metric': 'auc', 'learning_rate' : 0.01, 'max_depth':10, 'max_bin':10,  'objective': 'binary', 
          'feature_fraction': 0.8,'bagging_fraction':0.9,'bagging_freq':10,  'min_data': 500}
# local cv: 0.284789, lb: 0.281


In [8]:
# Cross Validation
# There has been some arguments on both sides for KFold and Stratified KFold. Need to investigate more.
# Discussion:
# @KALE I am now using Stratified 5Fold for my cv. But lgb cv score doesn't seem consistent with lb score. 
# lgb 0.282 - lb 0.279; lgb 0.281 - lb 0.280
# @鲲(China) 3 fold without Stratified is very consistent with lb in my side

# Personally, I feel that Stratified Kfold should be better, because the dataset has a imbalanced target, 
# and Stratified Kfold will try to balance out the different classes (0,1) in the target.

# https://www.kaggle.com/rshally/porto-xgb-lgb-kfold-lb-0-282   (Version 1)
nrounds=2000 
kfold = 5
skf = StratifiedKFold(n_splits=kfold, random_state=0)

In [9]:
# Actual Training
# 01 xgboost
# https://www.kaggle.com/rshally/porto-xgb-lgb-kfold-lb-0-282   (Version 1)
# Running time: ~45min

# About xgb_model.best_ntree_limit+50: 
# @Rudolph The credit for that goes to The1owl - it is not imperative but it seems to improve things a bit.
# @David Yang xgb_model.best_ntree_limit+50 seems to be unnecessary but it may get a good result.I think this way is useful to lgb, too. 
# You can change the +n/-n just like a parameter if it would imporove your result.
"""for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(' xgb kfold: {}  of  {} : '.format(i+1, kfold))
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    
    d_train = xgb.DMatrix(X_train, y_train) 
    d_valid = xgb.DMatrix(X_valid, y_valid) 
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    xgb_model = xgb.train(params, d_train, nrounds, watchlist, early_stopping_rounds=100, 
                          feval=gini_xgb, maximize=True, verbose_eval=100)
    prediction = xgb_model.predict(xgb.DMatrix(test[features].values), 
                        ntree_limit=xgb_model.best_ntree_limit+50) / (kfold)
    sub['target'] += prediction
    
gc.collect()
sub.head(2)"""

"for i, (train_index, test_index) in enumerate(skf.split(X, y)):\n    print(' xgb kfold: {}  of  {} : '.format(i+1, kfold))\n    X_train, X_valid = X[train_index], X[test_index]\n    y_train, y_valid = y[train_index], y[test_index]\n    \n    d_train = xgb.DMatrix(X_train, y_train) \n    d_valid = xgb.DMatrix(X_valid, y_valid) \n    watchlist = [(d_train, 'train'), (d_valid, 'valid')]\n    xgb_model = xgb.train(params, d_train, nrounds, watchlist, early_stopping_rounds=100, \n                          feval=gini_xgb, maximize=True, verbose_eval=100)\n    prediction = xgb_model.predict(xgb.DMatrix(test[features].values), \n                        ntree_limit=xgb_model.best_ntree_limit+50) / (kfold)\n    sub['target'] += prediction\n    \ngc.collect()\nsub.head(2)"

In [10]:
# 02 LightGBM
# https://www.kaggle.com/rshally/porto-xgb-lgb-kfold-lb-0-282   (Version 1)
# Running Time:
skf = StratifiedKFold(n_splits=kfold, random_state=1)
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(' lgb kfold: {}  of  {} : '.format(i+1, kfold))
    X_train, X_eval = X[train_index], X[test_index]
    y_train, y_eval = y[train_index], y[test_index]
    lgb_model = lgb.train(lgb_params, lgb.Dataset(X_train, label=y_train), nrounds, 
                  lgb.Dataset(X_eval, label=y_eval), verbose_eval=10, 
                  feval=gini_lgb, early_stopping_rounds=100)
    sub['target'] += lgb_model.predict(test[features].values, 
                        num_iteration=lgb_model.best_iteration) / (kfold)


 lgb kfold: 1  of  2 : 
Training until validation scores don't improve for 100 rounds.
[10]	valid_0's auc: 0.620841	valid_0's gini: 0.241691
[20]	valid_0's auc: 0.623424	valid_0's gini: 0.24685
[30]	valid_0's auc: 0.62626	valid_0's gini: 0.252521
[40]	valid_0's auc: 0.628462	valid_0's gini: 0.256924
[50]	valid_0's auc: 0.630364	valid_0's gini: 0.260728
[60]	valid_0's auc: 0.631282	valid_0's gini: 0.262564
[70]	valid_0's auc: 0.63218	valid_0's gini: 0.264361
[80]	valid_0's auc: 0.632385	valid_0's gini: 0.264771
[90]	valid_0's auc: 0.632935	valid_0's gini: 0.26587
[100]	valid_0's auc: 0.634087	valid_0's gini: 0.268174
[110]	valid_0's auc: 0.634799	valid_0's gini: 0.269598
[120]	valid_0's auc: 0.635491	valid_0's gini: 0.270983
[130]	valid_0's auc: 0.636165	valid_0's gini: 0.272331
[140]	valid_0's auc: 0.636787	valid_0's gini: 0.273575
[150]	valid_0's auc: 0.637315	valid_0's gini: 0.27463
[160]	valid_0's auc: 0.637797	valid_0's gini: 0.275595
[170]	valid_0's auc: 0.638091	valid_0's gini: 0

In [11]:
sub.to_csv('submission_5.csv', index=False, float_format='%.5f') 
gc.collect()
sub.head(2)

,id,target
0,0,0.057703
1,1,0.062681


In [12]:
sub.describe()

,id,target
count,8.928160e+05,892816.000000
mean,7.441535e+05,0.059406
std,4.296830e+05,0.019694
min,0.000000e+00,0.009830
25%,3.720218e+05,0.045099
50%,7.443070e+05,0.056896
75%,1.116308e+06,0.070877
max,1.488026e+06,0.183574


Insight:<br>
This kernel is mainly combining several kernels online and justifying every step inside, to help understand stacking better.